#### Exercise 8 Prediction Function

In [51]:
import librosa
import numpy as np
import pandas as pd
import os
import joblib
# # Added necessary imports for Pipeline components
# from sklearn.svm import SVC
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import TruncatedSVD
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split


In [52]:
# Required functions
def feature_vector(y, sr):
    """Return a 1D feature vector (mean/std over time for a few robust features)."""
    # MFCCs (20) + deltas (optional to keep minimal — comment out if you want even simpler)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)  # (20, T)
    mfcc_mean = mfcc.mean(axis=1); mfcc_std = mfcc.std(axis=1)

    # Basic spectral features (1 x T each)
    sc   = librosa.feature.spectral_centroid(y=y, sr=sr)
    sb   = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    ro   = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr  = librosa.feature.zero_crossing_rate(y)
    rms  = librosa.feature.rms(y=y)

    def stats(feat):  # (1, T) or (k, T) -> concat mean+std per row
        return np.hstack([feat.mean(axis=1), feat.std(axis=1)])

    dur = np.array([librosa.get_duration(y=y, sr=sr)], dtype=np.float32)

    x = np.hstack([
        mfcc_mean, mfcc_std,
        stats(sc), stats(sb), stats(ro), stats(zcr), stats(rms),
        dur
    ]).astype(np.float32)

    return x

In [53]:

def predict_sound_file(audio_path: str, pipeline, class_names: list) -> str:
    """
    Predicts the class label for a single sound file using the trained Scikit-learn Pipeline.

    Args:
        audio_path: Path to the WAV file.
        pipeline: The trained sklearn.pipeline.Pipeline object (Scaler -> SVD -> SVC).
        class_names: A list of class names, used for mapping the prediction index (not strictly needed 
                     if the pipeline's classes_ are used, but kept for clarity).

    Returns:
        The predicted class name as a string, or an error message.
    """
    print(f"\n--- Processing: {audio_path} ---")
    try:
        # 1. Load the Audio
        y, sr = librosa.load(audio_path, mono=True)

        if y is None or len(y) < 32:
            return "Error: File is too short or broken."

        # 2. Extract Features
        # Features must be the EXACT same dimensionality as the training data (51)
        feature_vec = feature_vector(y, sr)
        print(f"Features extracted. Shape: {feature_vec.shape}")

        # Features for prediction must be 2D (1 sample, N features)
        X_predict = feature_vec.reshape(1, -1)

        # 3. Predict using the Pipeline
        # The pipeline automatically handles scaling and SVD reduction before SVC classification
        predicted_classes = pipeline.predict(X_predict)
        predicted_class = predicted_classes[0]
        
        # Note: If the pipeline was trained on integer labels, you'd need the index.
        # Since the mock uses string labels, the output is directly the class name.

        return f"Predicted Class: {predicted_class}"

    except FileNotFoundError:
        return f"Error: File not found at path: {audio_path}"
    except Exception as e:
        return f"An error occurred during prediction: {e}"

In [54]:
prediction_data = '../Sounds_for_prediction/'
sounds = os.listdir(prediction_data)
TARGET_CLASSES = ["dog_bark", "street_music", "drilling"]

model = joblib.load('sound_model.pkl')

In [55]:
for i in sounds:
    i = prediction_data+i
    print(predict_sound_file(i,model, TARGET_CLASSES))


--- Processing: ../Sounds_for_prediction/dog_bark.mp3 ---
Features extracted. Shape: (51,)
Predicted Class: dog_bark

--- Processing: ../Sounds_for_prediction/street_carnival.mp3 ---
Features extracted. Shape: (51,)
Predicted Class: dog_bark

--- Processing: ../Sounds_for_prediction/power_drill.mp3 ---
Features extracted. Shape: (51,)
Predicted Class: drilling
